In [1]:
import pandas as pd
import numpy as np

import regex as re
import altair as alt

from datetime import datetime
from datetime import timedelta

cews = pd.read_csv("data/CEWS_with_subset_labels.csv")
# First column is just the index again
cews = cews.drop("Unnamed: 0", axis = 1)
# Datetime
cews.Period = pd.to_datetime(cews.Period, infer_datetime_format=True)
# Indicating intereset varaibles for analysis
num_cols = ['BusinessLocations', 'Subsidy', 'SupportedEmployees', 'RehiredEmployees']
cum_cols = ['BusinessLocations Cumulative', 'Subsidy Cumulative', 'SupportedEmployees Cumulative', 'RehiredEmployees Cumulative']
change_cols = ['BusinessLocations Change', 'Subsidy Change', 'SupportedEmployees Change', 'RehiredEmployees Change']
cumchange_cols = ['BusinessLocations Cumulative Change', 'Subsidy Cumulative Change', 'SupportedEmployees Cumulative Change', 'RehiredEmployees Cumulative Change']

FileNotFoundError: [Errno 2] No such file or directory: 'data/CEWS_with_subset_labels.csv'

In [ ]:
"This is Vicens first attempt at Visualizations. Nothing from this file was used in our final report"

# Dataframe Exploration

In [49]:
cews.head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation
0,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,11,"Agriculture, forestry, fishing and hunting",30.0,823000.0,362.0,0.0,Province,Level 1
1,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,111,Crop production,10.0,NaN,90.0,0.0,Province,Level 2
2,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,112,Animal production and aquaculture,10.0,NaN,NaN,0.0,Province,Level 2


## 1. Totals Period Analysis

In [50]:
temp = cews.query("GeoAggregation == 'Canada' and IndustryAggregation == 'All industries'").sort_values('Period')

# Adding cumulative columns
cumulative = temp.sort_values('Period')[num_cols].cumsum()
cumulative.columns = [str(col) + ' Cumulative' for col in cumulative.columns]
temp = temp.join(cumulative)
# Adding percent change columns
change = temp.sort_values('Period')[num_cols].pct_change()
change.columns = [str(col) + ' Change' for col in change.columns]
change = change.fillna(0)
temp = temp.join(change)
# Adding cumulative change columns
cumulative_change = temp.sort_values('Period')[change_cols].cumsum()
cumulative_change.columns = [str(col) + ' Change' for col in cumulative.columns]
temp = temp.join(cumulative_change)

temp.head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,...,SupportedEmployees Cumulative,RehiredEmployees Cumulative,BusinessLocations Change,Subsidy Change,SupportedEmployees Change,RehiredEmployees Change,BusinessLocations Cumulative Change,Subsidy Cumulative Change,SupportedEmployees Cumulative Change,RehiredEmployees Cumulative Change
71050,2020-03-15,TOTAL,Canada,Not applicable,Not applicable,TOTAL,All Industries,294070.0,7.459996e+09,3489214.0,...,3489214.0,30520.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
148158,2020-04-12,TOTAL,Canada,Not applicable,Not applicable,TOTAL,All Industries,336595.0,9.111729e+09,3743506.0,...,7232720.0,69465.0,0.144608,0.221412,0.072879,0.276048,0.144608,0.221412,0.072879,0.276048
225118,2020-05-10,TOTAL,Canada,Not applicable,Not applicable,TOTAL,All Industries,331865.0,9.647710e+09,4011166.0,...,11243886.0,113885.0,-0.014052,0.058823,0.071500,0.140583,0.130556,0.280235,0.144379,0.416631


In [51]:
df = temp

title = alt.Chart(df).mark_text(
            ).properties(title={'text': 'CEWS Evolution', 'subtitle': 'A negative trend (with different intensity) is visible in all interest variables',
                                "subtitleFontWeight": 900, "subtitleFontSize": 28})
evolution = alt.Chart(df).mark_bar(size=40).encode(
                x=alt.X('Period:T', title=None, scale=alt.Scale(domain=["2020-03-01", "2020-10-01"])),
                y=alt.Y(alt.repeat(), type='quantitative', 
                        axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
                tooltip=[alt.Tooltip('Period:T'),
                        alt.Tooltip(alt.repeat(), type='quantitative', format=',')]
                        ).repeat(num_cols, columns=2)
totals = (title & evolution).configure_title(fontSize=32, fontWeight=900, anchor='start'
                                   ).configure_axis(domain=False, grid=False, ticks=False
                                                   ).configure_view(stroke="transparent")

title2 = alt.Chart(df).mark_text(
            ).properties(title={'text': 'CEWS Cumulative Evolution', 'subtitle': 'Subsidy and rehired employees appear to be flattening',
                                "subtitleFontWeight": 900, "subtitleFontSize": 28})
evolution2 = alt.Chart(df).mark_area(point=True).encode(
                x=alt.X('Period:T', title=None, scale=alt.Scale(domain=["2020-03-15", "2020-10-01"])),
                y=alt.Y(alt.repeat(), type='quantitative', 
                        axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
                tooltip=[alt.Tooltip('Period:T'),
                        alt.Tooltip(alt.repeat(), type='quantitative', format=',')]
                        ).repeat(cum_cols, columns=2)
cumulative = (title2 & evolution2).configure_title(fontSize=32, fontWeight=900, anchor='start'
                                   ).configure_axis(domain=False, grid=False, ticks=False
                                                   ).configure_view(stroke="transparent")

title3 = alt.Chart(df).mark_text(
            ).properties(title={'text': 'CEWS Period Change', 'subtitle': 'Subsidy and rehired employees appear to be flattening',
                                "subtitleFontWeight": 900, "subtitleFontSize": 28})
evolution3 = alt.Chart(df).mark_bar(size=40).encode(
                x=alt.X('Period:T', title=None, scale=alt.Scale(domain=["2020-03-15", "2020-10-01"])),
                y=alt.Y(alt.repeat(), type='quantitative', 
                        axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='%')),
                tooltip=[alt.Tooltip('Period:T'),
                        alt.Tooltip(alt.repeat(), type='quantitative', format='.2%')]
                        ).repeat(change_cols, columns=2)
percent_change = (title3 & evolution3).configure_title(fontSize=32, fontWeight=900, anchor='start'
                                   ).configure_axis(domain=False, grid=False, ticks=False
                                                   ).configure_view(stroke="transparent")

title4 = alt.Chart(df).mark_text(
            ).properties(title={'text': 'CEWS Evolution', 'subtitle': 'Period, cumulative and average',
                                "subtitleFontWeight": 900, "subtitleFontSize": 28})
evolution4 = alt.Chart(df).mark_bar(size=40).encode(
                x=alt.X('Period:T', title=None, scale=alt.Scale(domain=["2020-03-01", "2020-10-01"])),
                y=alt.Y('Subsidy', type='quantitative', 
                        axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
                color = alt.Color('negative:N', legend=None, scale=alt.Scale(range=['lightblue', 'red'])),
                tooltip=[alt.Tooltip('Period:T'),
                        alt.Tooltip('Subsidy', type='quantitative', format=',')]
                        ).transform_calculate(negative= alt.datum['Subsidy'] < 0)
evolution5 = alt.Chart(df).mark_line(point=alt.OverlayMarkDef(filled=False, fill='red', size=100), size=5).encode(
                x=alt.X('Period:T', title=None, scale=alt.Scale(domain=["2020-03-15", "2020-10-01"])),
                y=alt.Y('Subsidy Cumulative', type='quantitative', 
                        axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
                tooltip=[alt.Tooltip('Period:T'),
                        alt.Tooltip('Subsidy Cumulative', type='quantitative', format=',')]
                        )
rule = alt.Chart(df).mark_text(dy=90, color='darkblue', size=40).encode(
        text=alt.Text('mean(Subsidy):Q', format=(','))) + alt.Chart(df
            ).mark_rule(color='darkblue', size=10
                ).encode(y='Subsidy):Q')
object1 = (evolution5 + rule)
subsidy = (evolution4 + object1).properties(width=800)
evolution6 = alt.Chart(df).mark_bar(size=40).encode(
                x=alt.X('Period:T', title=None, scale=alt.Scale(domain=["2020-03-01", "2020-10-01"])),
                y=alt.Y('Subsidy Change', type='quantitative', 
                        axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='%')),
                color = alt.Color('negative:N', legend=None, scale=alt.Scale(range=['lightblue', 'red'])),
                tooltip=[alt.Tooltip('Period:T'),
                        alt.Tooltip('Subsidy Change', type='quantitative', format='.2%')]
                        ).transform_calculate(negative= alt.datum['Subsidy Change'] < 0)
evolution7 = alt.Chart(df).mark_line(point=alt.OverlayMarkDef(filled=False, fill='red', size=100), size=5).encode(
                x=alt.X('Period:T', title=None, scale=alt.Scale(domain=["2020-03-15", "2020-10-01"])),
                y=alt.Y('Subsidy Cumulative Change', type='quantitative', 
                        axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
                tooltip=[alt.Tooltip('Period:T'),
                        alt.Tooltip('Subsidy Cumulative Change', type='quantitative', format='.2%')]
                        )
rule2 = alt.Chart(df).mark_text(dy=90, color='darkblue', size=40).encode(
        text=alt.Text('mean(Subsidy Change):Q', format=('.2%'))) + alt.Chart(df
            ).mark_rule(color='darkblue', size=10
                ).encode(y='Subsidy Change):Q')
object2 = (evolution7 + rule2)
subsidy_change = (evolution6 + object2).properties(width=800)
sub = (subsidy & subsidy_change)
subsidy_evo = (title4 & sub).configure_title(fontSize=32, fontWeight=900, anchor='start'
                                   ).configure_axis(domain=False, grid=False, ticks=False
                                                   ).configure_view(stroke="transparent")

graphs =[totals, cumulative, percent_change, subsidy_evo]

for i in range(len(graphs)):
    graphs[i].display()

alt.VConcatChart(...)

alt.VConcatChart(...)

alt.VConcatChart(...)

alt.VConcatChart(...)

Subsisy amount and rehired employees increased until peaking in the May 10 period. They have decreased since. \
Business locations and employees supported show a different behaviour, indicating that maybe the average subsidy decreased because the number of employees and business supported increased. \
The average subsidy per period is $6,950,515,375 and the average change per period is -10.08%

## 2. Industry Analysis

In [5]:
industry = cews.query("GeoAggregation == 'Province' and IndustryAggregation == 'Level 1'")
industry.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1906 entries, 0 to 562008
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Period                    1906 non-null   datetime64[ns]
 1   RegionCode                1906 non-null   object        
 2   Region                    1906 non-null   object        
 3   GeographicClassification  1906 non-null   object        
 4   CensusLevel               1906 non-null   object        
 5   IndustryCode              1906 non-null   object        
 6   Industry                  1906 non-null   object        
 7   BusinessLocations         1906 non-null   float64       
 8   Subsidy                   1523 non-null   float64       
 9   SupportedEmployees        1556 non-null   float64       
 10  RehiredEmployees          1906 non-null   float64       
 11  GeoAggregation            1906 non-null   object        
 12  IndustryAggregatio

I hoped that this query would be complete as it is at a provincial level and industry sector (lowest granularity).

In [6]:
industry[industry.isnull().any(axis=1)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 425 entries, 10 to 562008
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Period                    425 non-null    datetime64[ns]
 1   RegionCode                425 non-null    object        
 2   Region                    425 non-null    object        
 3   GeographicClassification  425 non-null    object        
 4   CensusLevel               425 non-null    object        
 5   IndustryCode              425 non-null    object        
 6   Industry                  425 non-null    object        
 7   BusinessLocations         425 non-null    float64       
 8   Subsidy                   42 non-null     float64       
 9   SupportedEmployees        75 non-null     float64       
 10  RehiredEmployees          425 non-null    float64       
 11  GeoAggregation            425 non-null    object        
 12  IndustryAggregatio

The query had a total of 425 surpressed rows. The null values represent 425/1906=22.30% of the total information.\

In [7]:
industry.Industry.unique()

array(['Agriculture, forestry, fishing and hunting',
       'Mining, quarrying, and oil and gas extraction', 'Utilities',
       'Construction', 'Manufacturing', 'Wholesale trade', 'Retail trade',
       'Transportation and warehousing',
       'Information and cultural industries', 'Finance and insurance',
       'Real estate and rental and leasing',
       'Professional, scientific and technical services',
       'Management of companies and enterprises',
       'Administrative and support, waste management and remediation services',
       'Educational services', 'Health care and social assistance',
       'Arts, entertainment and recreation',
       'Accommodation and food services', 'Other and Missing NAICS'],
      dtype=object)

In [8]:
# Checking the industries with incomplete information
industry[industry.isnull().any(axis=1)].Industry.unique()

array(['Utilities', 'Information and cultural industries',
       'Finance and insurance', 'Real estate and rental and leasing',
       'Management of companies and enterprises',
       'Administrative and support, waste management and remediation services',
       'Educational services',
       'Mining, quarrying, and oil and gas extraction',
       'Agriculture, forestry, fishing and hunting', 'Wholesale trade',
       'Professional, scientific and technical services',
       'Health care and social assistance',
       'Arts, entertainment and recreation', 'Other and Missing NAICS',
       'Accommodation and food services', 'Construction', 'Retail trade',
       'Manufacturing'], dtype=object)

In [9]:
industry[industry.Industry ==  'Manufacturing'].IndustryCode.unique()

array(['31-33'], dtype=object)

The only industry with complete information is the 'Transportation and warehousing'. I will explore it.

### 2.1. Transportation and Warehousing Analysis

In [10]:
temp1 = cews.query("GeoAggregation == 'Province' and Industry == 'Transportation and warehousing'")
temp2 = cews.query("Region == 'Canada' and Industry == 'Transportation and warehousing' and RegionCode == 'TOTAL'")
tran_ware = temp1.append(temp2)
mycols = ['Period', 'Region', 'BusinessLocations', 'Subsidy', 'SupportedEmployees', 'RehiredEmployees']
tran_ware = tran_ware[mycols]

# Adding cumulative columns
cumulative = tran_ware.sort_values('Period').groupby(['Region'])[num_cols].cumsum()
cumulative.columns = [str(col) + ' Cumulative' for col in cumulative.columns]
tran_ware = tran_ware.join(cumulative)
# Adding percent change columns
change = tran_ware.sort_values('Period').groupby(['Region'])[num_cols].pct_change()
change.columns = [str(col) + ' Change' for col in change.columns]
change = change.fillna(0)
tran_ware = tran_ware.join(change)
# Adding cumulative change columns
cumulative_change = tran_ware.sort_values('Period').groupby(['Region'])[change_cols].cumsum()
cumulative_change.columns = [str(col) + ' Change' for col in cumulative.columns]
tran_ware = tran_ware.join(cumulative_change)

tran_ware.head(3)

,Period,Region,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,BusinessLocations Cumulative,Subsidy Cumulative,SupportedEmployees Cumulative,RehiredEmployees Cumulative,BusinessLocations Change,Subsidy Change,SupportedEmployees Change,RehiredEmployees Change,BusinessLocations Cumulative Change,Subsidy Cumulative Change,SupportedEmployees Cumulative Change,RehiredEmployees Cumulative Change
56,2020-03-15,Newfoundland and Labrador,135.0,6509000.0,2485.0,10.0,135.0,6509000.0,2485.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2611,2020-03-15,Prince Edward Island,30.0,1430000.0,548.0,5.0,30.0,1430000.0,548.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3961,2020-03-15,Nova Scotia,195.0,11313000.0,4420.0,20.0,195.0,11313000.0,4420.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
title = alt.Chart(tran_ware).mark_text(
            ).properties(title={'text': 'Transportation and Warehousing Industry', 'subtitle': 'Province Breakdown',
                                "subtitleFontWeight": 900, "subtitleFontSize": 28})
chart = alt.Chart(tran_ware).mark_bar().encode(
            x=alt.X(alt.repeat(), type='quantitative'),
            y=alt.Y('Region:N', sort='-x'),
            tooltip=[alt.Tooltip('Region:N'),
                    alt.Tooltip(alt.repeat(), type='quantitative', format=',')]).repeat(cum_cols, columns=2)

(title & chart).configure_title(fontSize=32, fontWeight=900, anchor='start'
                                   ).configure_axis(domain=False, grid=False, ticks=False
                                                   ).configure_view(stroke="transparent")

alt.VConcatChart(...)

In [12]:
def custom_plot(data, category, variable, begin_date):
    
    df = data.groupby([category, 'Period'])[variable].sum().reset_index()
    df['Rate of change'] = df.sort_values('Period').groupby([category])[variable].pct_change()
    df.loc[df.Period == begin_date, 'Rate of change'] = 0
    df['Cumulative change'] = df.sort_values('Period').groupby([category])['Rate of change'].cumsum()

    cumulative = alt.Chart(df).mark_line(
                point=alt.OverlayMarkDef(filled=False, fill='red', size=75), size=5).encode(
                    x=alt.X('Period:T',
                        title=None,
                        axis=alt.Axis(tickCount=7, grid=False)),
                    y=alt.Y('sum(Cumulative change):Q',
                        title=None,
                    axis=alt.Axis(format='%', tickCount=4)),
                    tooltip=[alt.Tooltip('Period:T', format='%b %Y'), 
                             alt.Tooltip(field=category, type='nominal', title='Province'),
                             alt.Tooltip('sum(Cumulative change):Q', title='Cumulative change', format='.2%')])
    bar_pct = alt.Chart(df).mark_bar(size=15).encode(
            x=alt.X('Period:T',
                title=None,
                axis=alt.Axis(format='%Y', tickCount=7, grid=False)),
            y=alt.Y('sum(Rate of change):Q',
                title=None,
                axis=alt.Axis(format='%', tickCount=4)),
            color = alt.Color('negative:N', legend=None, scale=alt.Scale(range=['lightblue', 'red'])),
            tooltip=[alt.Tooltip('Period:T', title='Year', format='%Y'), 
                     alt.Tooltip(field=category, type='nominal', title='Province'), 
                     alt.Tooltip('sum(Rate of change):Q', title='Period change', format='.2%')]).transform_calculate(
                        negative= alt.datum['Rate of change'] < 0)
    rule = alt.Chart(df).mark_text(
        dy=90, color='darkblue', size=16).encode(
            text=alt.Text('mean(Rate of change):Q', format=('.2%'))) + alt.Chart(df).mark_rule(
                color='darkblue', size=3).encode(
                    y='mean(Rate of change):Q')
    ann_pct = (bar_pct + rule)
    chart = (cumulative + ann_pct).properties(height=170, width=130).facet(
                        facet=alt.Facet(field=category, type='nominal', sort=alt.EncodingSortField(field='Cumulative change', op='max', order='descending'), title='Period, cumulative and average rate of change comparison'), columns=5).resolve_scale(
                            y='independent').configure_axis(
                                domain=False, ticks=False).configure_view(strokeOpacity=0)
    return chart

In [13]:
data = tran_ware
category = 'Region'
variable = 'BusinessLocations'
begin_date = '2020-03-15'

custom_plot(data, category, variable, begin_date)

alt.FacetChart(...)

In [14]:
custom_plot(data, category, 'Subsidy', begin_date)

alt.FacetChart(...)

In [15]:
custom_plot(data, category, 'SupportedEmployees', begin_date)

alt.FacetChart(...)

In [16]:
custom_plot(data, category, 'RehiredEmployees', begin_date)

alt.FacetChart(...)

In [17]:
trans = cews[cews.IndustryCode.str.startswith('48')]
ware = cews[cews.IndustryCode.str.startswith('49')]
tw = trans.append(ware)
tw.GeoAggregation.unique()

array(['Province', 'urban/rural by province', 'CSD', 'CMA/CA',
       'Unknown Rural', 'Canada', 'urban/rural by country'], dtype=object)

In [18]:
aaa=tw.query("GeoAggregation == 'urban/rural by province' and IndustryAggregation == 'Level 1'").sort_values('Period')

In [19]:
aaa[aaa.isnull().any(axis=1)]

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation
69956,2020-03-15,60-urban,Yukon - urban part,URBAN,Not applicable,48-49,Transportation and warehousing,25.0,NaN,NaN,5.0,urban/rural by province,Level 1
69903,2020-03-15,60-rural,Yukon - rural part,RURAL,Not applicable,48-49,Transportation and warehousing,0.0,NaN,NaN,0.0,urban/rural by province,Level 1
2688,2020-03-15,11-rural,Prince Edward Island - rural part,RURAL,Not applicable,48-49,Transportation and warehousing,15.0,NaN,NaN,0.0,urban/rural by province,Level 1
2765,2020-03-15,11-urban,Prince Edward Island - urban part,URBAN,Not applicable,48-49,Transportation and warehousing,15.0,NaN,NaN,5.0,urban/rural by province,Level 1
147045,2020-04-12,60-urban,Yukon - urban part,URBAN,Not applicable,48-49,Transportation and warehousing,30.0,NaN,NaN,5.0,urban/rural by province,Level 1
146988,2020-04-12,60-rural,Yukon - rural part,RURAL,Not applicable,48-49,Transportation and warehousing,0.0,NaN,NaN,0.0,urban/rural by province,Level 1
74156,2020-04-12,11-rural,Prince Edward Island - rural part,RURAL,Not applicable,48-49,Transportation and warehousing,15.0,NaN,NaN,0.0,urban/rural by province,Level 1
74233,2020-04-12,11-urban,Prince Edward Island - urban part,URBAN,Not applicable,48-49,Transportation and warehousing,20.0,NaN,NaN,0.0,urban/rural by province,Level 1
561361,2020-09-27,60-rural,Yukon - rural part,RURAL,Not applicable,48-49,Transportation and warehousing,0.0,NaN,NaN,0.0,urban/rural by province,Level 1
561401,2020-09-27,60-urban,Yukon - urban part,URBAN,Not applicable,48-49,Transportation and warehousing,15.0,NaN,NaN,0.0,urban/rural by province,Level 1


In [20]:
tw.query("GeoAggregation == 'CSD' and IndustryAggregation == 'Level 1'").sort_values('Period')

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation
288,2020-03-15,1000000,Newfoundland and Labrador - rural part undeter...,RURAL,Not applicable,48-49,Transportation and warehousing,5.0,NaN,NaN,0.0,CSD,Level 1
46783,2020-03-15,4611042,Headingley,URBAN,Not applicable,48-49,Transportation and warehousing,5.0,165000.0,85.0,5.0,CSD,Level 1
46721,2020-03-15,4611040,Winnipeg,URBAN,Not applicable,48-49,Transportation and warehousing,175.0,7451000.0,3305.0,10.0,CSD,Level 1
46606,2020-03-15,4610035,Macdonald,URBAN,Not applicable,48-49,Transportation and warehousing,5.0,488000.0,185.0,0.0,CSD,Level 1
46550,2020-03-15,4608060,WestLake-Gladstone,RURAL,Not applicable,48-49,Transportation and warehousing,5.0,NaN,NaN,0.0,CSD,Level 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
523701,2020-09-27,2466097,Pointe-Claire,URBAN,Not applicable,48-49,Transportation and warehousing,15.0,117000.0,177.0,0.0,CSD,Level 1
523631,2020-09-27,2466087,Dorval,URBAN,Not applicable,48-49,Transportation and warehousing,50.0,7887000.0,4070.0,10.0,CSD,Level 1
523567,2020-09-27,2466072,Mont-Royal,URBAN,Not applicable,48-49,Transportation and warehousing,5.0,NaN,NaN,0.0,CSD,Level 1
523904,2020-09-27,2466142,Dollard-Des Ormeaux,URBAN,Not applicable,48-49,Transportation and warehousing,0.0,NaN,NaN,0.0,CSD,Level 1


In [21]:
tw.query("GeoAggregation == 'CMA/CA' and IndustryAggregation == 'Level 1'").sort_values('Period')

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation
340,2020-03-15,10001,St. John's,URBAN,CMA,48-49,Transportation and warehousing,65.0,3699000.0,1429.0,10.0,CMA/CA,Level 1
58859,2020-03-15,48810,Lethbridge,URBAN,CMA,48-49,Transportation and warehousing,35.0,744000.0,356.0,0.0,CMA/CA,Level 1
58781,2020-03-15,48806,Brooks,URBAN,CA,48-49,Transportation and warehousing,10.0,336000.0,114.0,0.0,CMA/CA,Level 1
58710,2020-03-15,48805,Medicine Hat,URBAN,CA,48-49,Transportation and warehousing,25.0,700000.0,271.0,0.0,CMA/CA,Level 1
52285,2020-03-15,47840,Lloydminster (Saskatchewan part),URBAN,CA,48-49,Transportation and warehousing,20.0,418000.0,139.0,0.0,CMA/CA,Level 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
518725,2020-09-27,24447,Drummondville,URBAN,CA,48-49,Transportation and warehousing,20.0,89000.0,110.0,0.0,CMA/CA,Level 1
518646,2020-09-27,24444,Shawinigan,URBAN,CA,48-49,Transportation and warehousing,5.0,54000.0,60.0,0.0,CMA/CA,Level 1
518573,2020-09-27,24442,Trois-Rivières,URBAN,CMA,48-49,Transportation and warehousing,35.0,298000.0,546.0,0.0,CMA/CA,Level 1
540159,2020-09-27,35555,London,URBAN,CMA,48-49,Transportation and warehousing,75.0,1301000.0,1639.0,5.0,CMA/CA,Level 1


In [22]:
tw.query("GeoAggregation == 'Unknown Rural' and IndustryAggregation == 'Level 1'").sort_values('Period')

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation
70003,2020-03-15,60000,Yukon - rural part excluding undetermined CSDs,RURAL,Not applicable,48-49,Transportation and warehousing,0.0,NaN,NaN,0.0,Unknown Rural,Level 1
70424,2020-03-15,61000,Northwest Territories - rural part excluding u...,RURAL,Not applicable,48-49,Transportation and warehousing,20.0,1631000.0,530.0,0.0,Unknown Rural,Level 1
70756,2020-03-15,62000,Nunavut - rural part excluding undetermined CSDs,RURAL,Not applicable,48-49,Transportation and warehousing,40.0,1879000.0,681.0,0.0,Unknown Rural,Level 1
147096,2020-04-12,60000,Yukon - rural part excluding undetermined CSDs,RURAL,Not applicable,48-49,Transportation and warehousing,0.0,NaN,NaN,0.0,Unknown Rural,Level 1
147535,2020-04-12,61000,Northwest Territories - rural part excluding u...,RURAL,Not applicable,48-49,Transportation and warehousing,20.0,1480000.0,480.0,0.0,Unknown Rural,Level 1
147872,2020-04-12,62000,Nunavut - rural part excluding undetermined CSDs,RURAL,Not applicable,48-49,Transportation and warehousing,40.0,1937000.0,683.0,0.0,Unknown Rural,Level 1
224089,2020-05-10,60000,Yukon - rural part excluding undetermined CSDs,RURAL,Not applicable,48-49,Transportation and warehousing,5.0,14000.0,5.0,0.0,Unknown Rural,Level 1
224526,2020-05-10,61000,Northwest Territories - rural part excluding u...,RURAL,Not applicable,48-49,Transportation and warehousing,25.0,1523000.0,516.0,5.0,Unknown Rural,Level 1
224846,2020-05-10,62000,Nunavut - rural part excluding undetermined CSDs,RURAL,Not applicable,48-49,Transportation and warehousing,40.0,1928000.0,763.0,10.0,Unknown Rural,Level 1
296211,2020-06-07,61000,Northwest Territories - rural part excluding u...,RURAL,Not applicable,48-49,Transportation and warehousing,25.0,1529000.0,501.0,5.0,Unknown Rural,Level 1


In [23]:
tw.query("GeoAggregation == 'urban/rural by country' and IndustryAggregation == 'Level 1'").sort_values('Period')

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation
71111,2020-03-15,rural,Canada,RURAL,Not applicable,48-49,Transportation and warehousing,2025.0,50968000.0,20700.0,110.0,urban/rural by country,Level 1
71218,2020-03-15,urban,Canada,URBAN,Not applicable,48-49,Transportation and warehousing,7410.0,379694000.0,150597.0,585.0,urban/rural by country,Level 1
148219,2020-04-12,rural,Canada,RURAL,Not applicable,48-49,Transportation and warehousing,2665.0,64432000.0,25076.0,190.0,urban/rural by country,Level 1
148326,2020-04-12,urban,Canada,URBAN,Not applicable,48-49,Transportation and warehousing,9390.0,467714000.0,176494.0,745.0,urban/rural by country,Level 1
225179,2020-05-10,rural,Canada,RURAL,Not applicable,48-49,Transportation and warehousing,2680.0,70329000.0,27916.0,270.0,urban/rural by country,Level 1
225286,2020-05-10,urban,Canada,URBAN,Not applicable,48-49,Transportation and warehousing,9090.0,476047000.0,181522.0,900.0,urban/rural by country,Level 1
296848,2020-06-07,rural,Canada,RURAL,Not applicable,48-49,Transportation and warehousing,2320.0,62909000.0,24858.0,185.0,urban/rural by country,Level 1
296955,2020-06-07,urban,Canada,URBAN,Not applicable,48-49,Transportation and warehousing,7720.0,415981000.0,158233.0,730.0,urban/rural by country,Level 1
368577,2020-07-05,rural,Canada,RURAL,Not applicable,48-49,Transportation and warehousing,2345.0,53107000.0,26217.0,105.0,urban/rural by country,Level 1
368684,2020-07-05,urban,Canada,URBAN,Not applicable,48-49,Transportation and warehousing,7965.0,374784000.0,178651.0,490.0,urban/rural by country,Level 1


# May 13th, 2021

Cheching the number of businesses supported vs the total number of businesses by province to see which industry was most hit.

In [24]:
# Creating a dataframe with the total number of businesses. This dataset is taken from StatCan and available in our repo.
bcounts = pd.read_csv("more_data/businessCountByProvince.csv")

bcounts.rename(columns={'GEO': 'Region', 'North American Industry Classification System (NAICS)' : 'Industry', 'VALUE' : 'Total Businesses'}, inplace=True)
dummy = bcounts.Industry.str.split('[', n=1, expand=True)
bcounts['IndustryCode'] = dummy[1].str.split(']').str[0]
bcounts.Industry = dummy[0].str.rstrip()

bcounts.head(3)

,Region,Industry,Total Businesses,IndustryCode
0,Canada,"Agriculture, forestry, fishing and hunting",49347,11
1,Canada,"Mining, quarrying, and oil and gas extraction",8383,21
2,Canada,Utilities,1431,22


In [25]:
# Querying provinces and level 1 industries with totals to perform the calculation
one = cews.query("GeoAggregation == 'Province' and IndustryAggregation == 'Level 1' and Period == '2020-03-15'")
two = cews.query("Region == 'Canada' and IndustryAggregation == 'Level 1' and Period == '2020-03-15' and RegionCode == 'TOTAL'")
prov_ind = one.append(two).groupby(['Region', 'Industry']).sum().reset_index()

prov_ind.head(3)

,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees
0,Alberta,Accommodation and food services,4470.0,80660000.0,66393.0,695.0
1,Alberta,"Administrative and support, waste management a...",1660.0,54073000.0,23031.0,140.0
2,Alberta,"Agriculture, forestry, fishing and hunting",385.0,7938000.0,3347.0,10.0


In [26]:
# Merging the dataframes
prov_ind = pd.merge(prov_ind, bcounts,  how='right', on=['Region','Industry'])

#Calculating the ratio of total supported businesses
prov_ind['Supported Ratio'] = prov_ind['BusinessLocations'] / prov_ind['Total Businesses']

prov_ind.head(3)

,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,Total Businesses,IndustryCode,Supported Ratio
0,Canada,"Agriculture, forestry, fishing and hunting",3730.0,78101000.0,35240.0,165.0,49347,11,0.075587
1,Canada,"Mining, quarrying, and oil and gas extraction",2045.0,251298000.0,80631.0,80.0,8383,21,0.243946
2,Canada,Utilities,105.0,13719000.0,4234.0,0.0,1431,22,0.073375


**Most supported industries by (business supported / total business) ratio**

In [27]:
prov_ind[prov_ind.Region == 'Canada'].sort_values('Supported Ratio', ascending=False)

,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,Total Businesses,IndustryCode,Supported Ratio
17,Canada,Accommodation and food services,33770.0,5.263000e+08,474363.0,5900.0,83558,72,0.404150
16,Canada,"Arts, entertainment and recreation",6640.0,1.446340e+08,84239.0,590.0,18947,71,0.350451
4,Canada,Manufacturing,16950.0,1.233185e+09,485250.0,1725.0,51179,31-33,0.331191
6,Canada,Retail trade,46470.0,7.153080e+08,420209.0,6385.0,142571,44-45,0.325943
8,Canada,Information and cultural industries,5985.0,2.045140e+08,84668.0,425.0,18601,51,0.321757
14,Canada,Educational services,4770.0,1.098900e+08,55603.0,490.0,15259,61,0.312602
5,Canada,Wholesale trade,17050.0,6.242730e+08,245753.0,1405.0,56420,41,0.302198
12,Canada,Management of companies and enterprises,1775.0,2.552310e+08,95029.0,185.0,6632,55,0.267642
1,Canada,"Mining, quarrying, and oil and gas extraction",2045.0,2.512980e+08,80631.0,80.0,8383,21,0.243946
15,Canada,Health care and social assistance,30035.0,3.669070e+08,192079.0,3470.0,126450,62,0.237525


In [28]:
alt.Chart(prov_ind[prov_ind.Region == 'Canada']).mark_bar().encode(
    y=alt.Y('Industry:N', title='Industry Sector', sort='-x'),
    x=alt.X('Supported Ratio:Q', title='% of Businesses Suported', scale=alt.Scale(domain=(0,1)), axis=alt.Axis(format='%'))
    ).properties(title={'text': 'Proportion of Business Supported by Industry Sector', 'subtitle': '1st 4-week period (March 15, 2020)'})

alt.Chart(...)

The accomodation and food services was the most suported by this ratio. I will explore it further.

In [29]:
# Creating a dataframe with all the claims periods
one = cews.query("GeoAggregation == 'Province' and Industry == 'Accommodation and food services'")
two = cews.query("Region == 'Canada' and Industry == 'Accommodation and food services' and RegionCode == 'TOTAL'")
acco = one.append(two).groupby(['Period', 'Region', 'Industry']).sum().reset_index()

three = cews.query("GeoAggregation == 'Province' and Industry == 'Arts, entertainment and recreation'")
four = cews.query("Region == 'Canada' and Industry == 'Arts, entertainment and recreation' and RegionCode == 'TOTAL'")
arts = three.append(four).groupby(['Period', 'Region', 'Industry']).sum().reset_index()

acco.head(3)

,Period,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees
0,2020-03-15,Alberta,Accommodation and food services,4470.0,80660000.0,66393.0,695.0
1,2020-03-15,British Columbia,Accommodation and food services,6065.0,102021000.0,84775.0,1150.0
2,2020-03-15,Canada,Accommodation and food services,33770.0,526300000.0,474363.0,5900.0


In [30]:
arts.head(3)

,Period,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees
0,2020-03-15,Alberta,"Arts, entertainment and recreation",710.0,24863000.0,15163.0,55.0
1,2020-03-15,British Columbia,"Arts, entertainment and recreation",1175.0,23697000.0,13099.0,115.0
2,2020-03-15,Canada,"Arts, entertainment and recreation",6640.0,144634000.0,84239.0,590.0


In [31]:
alt.Chart(acco[acco.Region == 'Canada']).mark_bar(size=30).encode(
    y=alt.Y('BusinessLocations:Q', title='Numer of Busineess Locations Supported'),
    x=alt.X('Period:T', title='4-week Claim Period', axis=alt.Axis(tickCount=8, format=('%b %Y')), scale=alt.Scale(domain = ['2020-03-01', '2020-10-01']))
    ).properties(title={'text': 'Accomodation and Food Services', 'subtitle': 'Business Supported Evolution'})

alt.Chart(...)

In [32]:
alt.Chart(arts[arts.Region == 'Canada']).mark_bar(size=30).encode(
    y=alt.Y('BusinessLocations:Q', title='Numer of Busineess Locations Supported'),
    x=alt.X('Period:T', title='4-week Claim Period', axis=alt.Axis(tickCount=8, format=('%b %Y')), scale=alt.Scale(domain = ['2020-03-01', '2020-10-01'])),
    tooltip=[alt.Tooltip('BusinessLocations:Q')]
    ).properties(title={'text': 'Arts, entertainment and recreation', 'subtitle': 'Business Supported Evolution'})

alt.Chart(...)

**Most supported provinces by (business supported / total business) ratio in the accomodation and food services industry**

In [33]:
prov_ind[prov_ind.Industry == 'Accommodation and food services'].sort_values('Supported Ratio', ascending=False)

,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,Total Businesses,IndustryCode,Supported Ratio
197,British Columbia,Accommodation and food services,6065.0,102021000.0,84775.0,1150.0,13307,72,0.455775
179,Alberta,Accommodation and food services,4470.0,80660000.0,66393.0,695.0,9894,72,0.451789
161,Saskatchewan,Accommodation and food services,1165.0,15239000.0,15372.0,175.0,2707,72,0.430366
125,Ontario,Accommodation and food services,13000.0,207811000.0,182368.0,2360.0,30521,72,0.425936
143,Manitoba,Accommodation and food services,1040.0,17477000.0,18219.0,170.0,2500,72,0.416000
89,New Brunswick,Accommodation and food services,700.0,9406000.0,9198.0,90.0,1695,72,0.412979
71,Nova Scotia,Accommodation and food services,895.0,11872000.0,12374.0,140.0,2178,72,0.410927
17,Canada,Accommodation and food services,33770.0,526300000.0,474363.0,5900.0,83558,72,0.404150
35,Newfoundland and Labrador,Accommodation and food services,470.0,7760000.0,6557.0,50.0,1231,72,0.381803
233,Northwest Territories,Accommodation and food services,45.0,0.0,439.0,5.0,119,72,0.378151


In [34]:
    alt.Chart(prov_ind[prov_ind.Industry == 'Accommodation and food services']).mark_bar().encode(
        y=alt.Y('Region:N',  title=None, sort='-x'),
    x=alt.X('Supported Ratio:Q', title='% of Businesses Suported', scale=alt.Scale(domain=(0,1)), axis=alt.Axis(format='%'))
    ).properties(title={'text': 'Accommodation and food services Proportion of Business Supported by Province/Territory', 'subtitle': '1st 4-week period (March 15, 2020)'})

alt.Chart(...)

Checking the number of businesses supported vs the total number of businesses by province to see which subindustry in the Accomodation and Food Serices was most hit.

In [35]:
# Creating a dataframe with the total number of businesses. This dataset is taken from StatCan and available in our repo.
bcounts2 = pd.read_csv("more_data/AccommoddationFoodbusinessCountByProvince.csv")

bcounts2.rename(columns={'GEO': 'Region', 'North American Industry Classification System (NAICS)' : 'Industry', 'VALUE' : 'Total Businesses'}, inplace=True)
dummy = bcounts2.Industry.str.split('[', n=1, expand=True)
bcounts2['IndustryCode'] = dummy[1].str.split(']').str[0]
bcounts2.Industry = dummy[0].str.rstrip()

bcounts2.head(3)

,Region,Industry,Total Businesses,IndustryCode
0,Canada,Accommodation and food services,83558,72
1,Canada,Accommodation services,10507,721
2,Canada,Food services and drinking places,73051,722


In [36]:
# Creating a dataframe for the accomodation and food services
acco_and_food = cews[cews.IndustryCode.str.startswith('72')]

acco_and_food.head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation
94,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,72,Accommodation and food services,470.0,7760000.0,6557.0,50.0,Province,Level 1
95,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,721,Accommodation services,85.0,1696000.0,835.0,10.0,Province,Level 2
96,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,722,Food services and drinking places,385.0,6064000.0,5722.0,40.0,Province,Level 2


In [37]:
# Querying provinces and level 2 industries with totals to perform the calculation
one = acco_and_food.query("GeoAggregation == 'Province' and IndustryAggregation == 'Level 2' and Period == '2020-03-15'")
two = acco_and_food.query("Region == 'Canada' and IndustryAggregation == 'Level 2' and Period == '2020-03-15' and RegionCode == 'TOTAL'")
acco_and_food_prov_subind = one.append(two).groupby(['Region', 'Industry']).sum().reset_index()

acco_and_food_prov_subind.head(3)

,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees
0,Alberta,Accommodation services,640.0,20364000.0,13530.0,70.0
1,Alberta,Food services and drinking places,3830.0,60296000.0,52864.0,625.0
2,British Columbia,Accommodation services,860.0,24577000.0,15324.0,70.0


In [38]:
# Merging the dataframes
acco_and_food_prov_subind = pd.merge(acco_and_food_prov_subind, bcounts2,  how='right', on=['Region','Industry'])

#Calculating the ratio of total supported businesses
acco_and_food_prov_subind['Supported Ratio'] = acco_and_food_prov_subind['BusinessLocations'] / acco_and_food_prov_subind['Total Businesses']

acco_and_food_prov_subind.head(3)

,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,Total Businesses,IndustryCode,Supported Ratio
0,Canada,Accommodation and food services,NaN,NaN,NaN,NaN,83558,72,NaN
1,Canada,Accommodation services,4120.0,107510000.0,72122.0,430.0,10507,721,0.392120
2,Canada,Food services and drinking places,29650.0,418790000.0,402242.0,5470.0,73051,722,0.405881


**Most supported provinces by (business supported / total business) ratio in the accomodation services industry**

In [39]:
acco_and_food_prov_subind[acco_and_food_prov_subind.Industry == 'Accommodation services'].sort_values('Supported Ratio', ascending=False)

,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,Total Businesses,IndustryCode,Supported Ratio
28,Alberta,Accommodation services,640.0,20364000.0,13530.0,70.0,1288,721,0.496894
31,British Columbia,Accommodation services,860.0,24577000.0,15324.0,70.0,1917,721,0.448618
19,Ontario,Accommodation services,1080.0,35646000.0,22816.0,100.0,2636,721,0.409712
22,Manitoba,Accommodation services,180.0,4585000.0,4166.0,15.0,448,721,0.401786
1,Canada,Accommodation services,4120.0,107510000.0,72122.0,430.0,10507,721,0.392120
25,Saskatchewan,Accommodation services,220.0,3654000.0,2811.0,20.0,577,721,0.381282
34,Yukon,Accommodation services,30.0,0.0,0.0,5.0,85,721,0.352941
13,New Brunswick,Accommodation services,115.0,1314000.0,1139.0,10.0,330,721,0.348485
40,Nunavut,Accommodation services,10.0,0.0,76.0,0.0,29,721,0.344828
16,Quebec,Accommodation services,730.0,12428000.0,8994.0,120.0,2217,721,0.329274


In [40]:
alt.Chart(acco_and_food_prov_subind[acco_and_food_prov_subind.Industry == 'Accommodation services']).mark_bar().encode(
    y=alt.Y('Region:N',  title=None, sort='-x'),
    x=alt.X('Supported Ratio:Q', title='% of Businesses Suported', scale=alt.Scale(domain=(0,1)), axis=alt.Axis(format='%'))
    ).properties(title={'text': 'Accommodation services Proportion of Business Supported by Province/Territory', 'subtitle': '1st 4-week period (March 15, 2020)'})

alt.Chart(...)

**Most supported provinces by (business supported / total business) ratio in the food services and drinking places industry**

In [41]:
acco_and_food_prov_subind[acco_and_food_prov_subind.Industry == 'Food services and drinking places'].sort_values('Supported Ratio', ascending=False)

,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,Total Businesses,IndustryCode,Supported Ratio
32,British Columbia,Food services and drinking places,5205.0,77444000.0,69451.0,1080.0,11390,722,0.456980
29,Alberta,Food services and drinking places,3830.0,60296000.0,52864.0,625.0,8606,722,0.445038
26,Saskatchewan,Food services and drinking places,945.0,11585000.0,12561.0,155.0,2130,722,0.443662
5,Newfoundland and Labrador,Food services and drinking places,385.0,6064000.0,5722.0,40.0,871,722,0.442021
11,Nova Scotia,Food services and drinking places,785.0,9903000.0,10874.0,135.0,1780,722,0.441011
38,Northwest Territories,Food services and drinking places,25.0,0.0,0.0,0.0,58,722,0.431034
14,New Brunswick,Food services and drinking places,585.0,8092000.0,8059.0,80.0,1365,722,0.428571
20,Ontario,Food services and drinking places,11920.0,172165000.0,159552.0,2260.0,27885,722,0.427470
23,Manitoba,Food services and drinking places,860.0,12891000.0,14053.0,155.0,2052,722,0.419103
2,Canada,Food services and drinking places,29650.0,418790000.0,402242.0,5470.0,73051,722,0.405881


In [42]:
alt.Chart(acco_and_food_prov_subind[acco_and_food_prov_subind.Industry == 'Food services and drinking places']).mark_bar().encode(
    y=alt.Y('Region:N',  title=None, sort='-x'),
    x=alt.X('Supported Ratio:Q', title='% of Businesses Suported', scale=alt.Scale(domain=(0,1)), axis=alt.Axis(format='%'))
    ).properties(title={'text': 'Food services and drinking places Proportion of Business Supported by Province/Territory', 'subtitle': '1st 4-week period (March 15, 2020)'})

alt.Chart(...)

I will dig deeper into Alberta accomodation services subsector industry as 1 out 2 businessses were supported at the start of the CEWS program.

In [43]:
one = acco_and_food.query("GeoAggregation == 'Province' and Industry == 'Accommodation services'")
two = acco_and_food.query("Region == 'Canada' and Industry == 'Accommodation services' and RegionCode == 'TOTAL'")
acco_and_food_prov_subind = one.append(two).groupby(['Period','Region', 'Industry']).sum().reset_index()

acco_and_food_prov_subind.head(3)

,Period,Region,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees
0,2020-03-15,Alberta,Accommodation services,640.0,20364000.0,13530.0,70.0
1,2020-03-15,British Columbia,Accommodation services,860.0,24577000.0,15324.0,70.0
2,2020-03-15,Canada,Accommodation services,4120.0,107510000.0,72122.0,430.0


In [44]:
alt.Chart(acco_and_food_prov_subind[acco_and_food_prov_subind.Region == 'Alberta']).mark_bar(size=30).encode(
    y=alt.Y('BusinessLocations:Q', title='Numer of Busineess Locations Supported'),
    x=alt.X('Period:T', title='4-week Claim Period', axis=alt.Axis(tickCount=8, format=('%b %Y')), scale=alt.Scale(domain = ['2020-03-01', '2020-10-01'])),
    tooltip=[alt.Tooltip('BusinessLocations:Q')]
    ).properties(title={'text': 'Alberta Accomodation Services', 'subtitle': 'Business Supported Evolution'})

alt.Chart(...)

In [45]:
alt.Chart(acco_and_food_prov_subind[acco_and_food_prov_subind.Region == 'Alberta']).mark_bar(size=30).encode(
    y=alt.Y('Subsidy:Q', title='Total Subsidy'),
    x=alt.X('Period:T', title='4-week Claim Period', axis=alt.Axis(tickCount=8, format=('%b %Y')), scale=alt.Scale(domain = ['2020-03-01', '2020-10-01'])),
    tooltip=[alt.Tooltip('BusinessLocations:Q')]
    ).properties(title={'text': 'Alberta Accomodation Services', 'subtitle': 'Total Subsidy Evolution'})

alt.Chart(...)

In [46]:
alt.Chart(acco_and_food_prov_subind[acco_and_food_prov_subind.Region == 'Alberta']).mark_bar(size=30).encode(
    y=alt.Y('SupportedEmployees:Q', title='Number of Employees Supported'),
    x=alt.X('Period:T', title='4-week Claim Period', axis=alt.Axis(tickCount=8, format=('%b %Y')), scale=alt.Scale(domain = ['2020-03-01', '2020-10-01'])),
    tooltip=[alt.Tooltip('BusinessLocations:Q')]
    ).properties(title={'text': 'Alberta Accomodation Services', 'subtitle': 'Employees Supported Evolution'})

alt.Chart(...)

In [47]:
alt.Chart(acco_and_food_prov_subind[acco_and_food_prov_subind.Region == 'Alberta']).mark_bar(size=30).encode(
    y=alt.Y('RehiredEmployees:Q', title='Numer of Rehired Employees'),
    x=alt.X('Period:T', title='4-week Claim Period', axis=alt.Axis(tickCount=8, format=('%b %Y')), scale=alt.Scale(domain = ['2020-03-01', '2020-10-01'])),
    tooltip=[alt.Tooltip('BusinessLocations:Q')]
    ).properties(title={'text': 'Alberta Accomodation Services', 'subtitle': 'Rehired Evolution'})

alt.Chart(...)